In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers as esHelper
import json



In [2]:
esconfig = {
    "host": [
        {
            "host": "localhost",                  
            "port":"9229"
        }
    ],
    "log": "error"
}

In [3]:
es = Elasticsearch( hosts= esconfig['host'])

In [5]:
es.info()
#es.indices.delete(index='imdb', ignore=[400, 404])
es.nodes.info()

{'_nodes': {'failed': 0, 'successful': 1, 'total': 1},
 'cluster_name': 'cluster-es',
 'nodes': {'F-Wp8wX3Q_qYkIHZZO3AJA': {'attributes': {'ml.enabled': 'true',
    'ml.machine_memory': '31615455232',
    'ml.max_open_jobs': '20'},
   'build_hash': '10b1edd',
   'host': '10.64.54.215',
   'http': {'bound_address': ['[::1]:9229',
     '127.0.0.1:9229',
     '172.28.46.232:9229',
     '10.64.54.215:9229'],
    'max_content_length_in_bytes': 1073741824,
    'publish_address': '10.64.54.215:9229'},
   'ingest': {'processors': [{'type': 'append'},
     {'type': 'convert'},
     {'type': 'date'},
     {'type': 'date_index_name'},
     {'type': 'dot_expander'},
     {'type': 'fail'},
     {'type': 'foreach'},
     {'type': 'grok'},
     {'type': 'gsub'},
     {'type': 'join'},
     {'type': 'json'},
     {'type': 'kv'},
     {'type': 'lowercase'},
     {'type': 'remove'},
     {'type': 'rename'},
     {'type': 'script'},
     {'type': 'set'},
     {'type': 'set_security_user'},
     {'type': 

In [ ]:
es.indices.delete(index='imdb', ignore=[400, 404])
es.indices.create(index='imdb', ignore=400)
es.indices.put_mapping(index='imdb', doc_type='titles', body=json.load(open('./data/imdb.title.mapping.json')))

In [ ]:
es.indices.get_mapping(index='imdb')

In [ ]:
def prep_bulk_data(data):
            for item in data:                
                yield {
                       '_index': 'imdb',
                       '_type': 'titles',
                       '_id': item['tconst'],
                       '_source': item
                }
                
for ok, result in esHelper.streaming_bulk(es, prep_bulk_data(json.load(open('./data/titles.1.json')))):
            if not ok:
                __, result = result.popitem()
                if result['status'] == 409:
                    print('Duplicate event detected, skipping it: {}'.format (result))
                else:
                    print('Failed to record event: {}'.format(result))
            #else:
               # print('Success {}'.format(result))
 
       

In [ ]:
from helper.es_helper import ESHelper
esHelper = ESHelper(esconfig)

In [ ]:
esHelper.info()

In [1]:
import json
def prep_bulk_data(data):
            for item in data:                
                yield {
                       '_index': 'imdb',
                       '_type': 'titles',
                       '_id': item['tconst'],
                       '_source': item
                }

#esHelper.stream_bulk(prep_bulk_data(json.load(open('./data/titles.0.json'))))
prep_bulk_data(json.load(open('./data/titles.0.json')))

<generator object prep_bulk_data at 0x7f57b42c8150>